In [1]:
import tkinter as tk
from tkinter import *
from tkinter import filedialog
from cv2 import imread, imwrite
import numpy as np

In [21]:
import cv2
import numpy as np

In [1]:
#project 
import cv2
import numpy as np

def to_bin(data):
    """Convert `data` to binary format as string"""
    if isinstance(data, str):
        return ''.join([ format(ord(i), "08b") for i in data ])
    elif isinstance(data, bytes):
        return ''.join([ format(i, "08b") for i in data ])
    elif isinstance(data, np.ndarray):
        return [ format(i, "08b") for i in data ]
    elif isinstance(data, int) or isinstance(data, np.uint8):
        return format(data, "08b")
    else:
        raise TypeError("Type not supported.")
    
def encode(image_name, secret_data):
    # read the image
    image = cv2.imread(image_name)
    # maximum bytes to encode
    n_bytes = image.shape[0] * image.shape[1] * 3 // 8
    print("[*] Maximum bytes to encode:", n_bytes)
    if len(secret_data) > n_bytes:
        raise ValueError("[!] Insufficient bytes, need bigger image or less data.")
    print("[*] Encoding data...")
    # add stopping criteria
    secret_data += "====="
    data_index = 0
    # convert data to binary
    binary_secret_data = to_bin(secret_data)
    # size of data to hide
    data_len = len(binary_secret_data)
    for row in image:
        for pixel in row:
            # convert RGB values to binary format
            r, g, b = to_bin(pixel)
            # modify the least significant bit only if there is still data to store
            if data_index < data_len:
                # least significant red pixel bit
                pixel[0] = int(r[:-1] + binary_secret_data[data_index], 2)
                data_index += 1
            if data_index < data_len:
                # least significant green pixel bit
                pixel[1] = int(g[:-1] + binary_secret_data[data_index], 2)
                data_index += 1
            if data_index < data_len:
                # least significant blue pixel bit
                pixel[2] = int(b[:-1] + binary_secret_data[data_index], 2)
                data_index += 1
            # if data is encoded, just break out of the loop
            if data_index >= data_len:
                break
    return image
def decode(image_name):
    print("[+] Decoding...")
    # read the image
    image = cv2.imread(image_name)
    binary_data = ""
    for row in image:
        for pixel in row:
            r, g, b = to_bin(pixel)
            binary_data += r[-1]
            binary_data += g[-1]
            binary_data += b[-1]
    # split by 8-bits
    all_bytes = [ binary_data[i: i+8] for i in range(0, len(binary_data), 8) ]
    # convert from bits to characters
    decoded_data = ""
    for byte in all_bytes:
        decoded_data += chr(int(byte, 2))
        if decoded_data[-5:] == "=====":
            break
    return decoded_data[:-5]  

if __name__ == "__main__":
    input_image = "img/1920RepublicanPresidentialNomination9thBallot.png"
    output_image = "encoded_image.PNG"
    secret_data = "This is a top secret message."
    # encode the data into the image
    encoded_image = encode(image_name=input_image, secret_data=secret_data)
    # save the output image (encoded image)
    cv2.imwrite(output_image, encoded_image)
    # decode the secret data from the image
    decoded_data = decode(output_image)
    print("[+] Decoded data:", decoded_data) 
 

[*] Maximum bytes to encode: 59350
[*] Encoding data...
[+] Decoding...
[+] Decoded data: This is a top secret message.


In [51]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import ttk, filedialog

def to_bin(data):
    if isinstance(data, str):
        return ''.join([format(ord(i), "08b") for i in data])
    elif isinstance(data, bytes):
        return ''.join([format(i, "08b") for i in data])
    elif isinstance(data, np.ndarray):
        return [format(i, "08b") for i in data]
    elif isinstance(data, int) or isinstance(data, np.uint8):
        return format(data, "08b")
    else:
        raise TypeError("Type not supported.")

def encode(image_name, secret_data):
    image = cv2.imread(image_name)
    n_bytes = image.shape[0] * image.shape[1] * 3 // 8
    if len(secret_data) > n_bytes:
        raise ValueError("Insufficient bytes, need bigger image or less data.")
    secret_data += "====="
    data_index = 0
    binary_secret_data = to_bin(secret_data)
    data_len = len(binary_secret_data)
    for row in image:
        for pixel in row:
            r, g, b = to_bin(pixel)
            if data_index < data_len:
                pixel[0] = int(r[:-1] + binary_secret_data[data_index], 2)
                data_index += 1
            if data_index < data_len:
                pixel[1] = int(g[:-1] + binary_secret_data[data_index], 2)
                data_index += 1
            if data_index < data_len:
                pixel[2] = int(b[:-1] + binary_secret_data[data_index], 2)
                data_index += 1
            if data_index >= data_len:
                break
    return image

def decode(image_name):
    image = cv2.imread(image_name)
    binary_data = ""
    for row in image:
        for pixel in row:
            r, g, b = to_bin(pixel)
            binary_data += r[-1]
            binary_data += g[-1]
            binary_data += b[-1]
    all_bytes = [binary_data[i: i + 8] for i in range(0, len(binary_data), 8)]
    decoded_data = ""
    for byte in all_bytes:
        decoded_data += chr(int(byte, 2))
        if decoded_data[-5:] == "=====":
            break
    return decoded_data[:-5]

def browse_image(entry_widget):
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
    entry_widget.delete(0, tk.END)
    entry_widget.insert(0, file_path)

def encode_data():
    input_image = entry_input_image.get()
    output_image = entry_output_image.get()
    secret_data = entry_secret_data.get("1.0", tk.END).strip()

    if input_image and output_image and secret_data:
        encoded_image = encode(image_name=input_image, secret_data=secret_data)
        cv2.imwrite(output_image, encoded_image)
        tk.messagebox.showinfo("Encoding", "Data has been successfully encoded.")
    else:
        tk.messagebox.showerror("Error", "Please fill in all fields.")

def decode_data():
    input_image = entry_input_image.get()

    if input_image:
        decoded_data = decode(image_name=input_image)
        entry_secret_data.delete("1.0", tk.END)
        entry_secret_data.insert(tk.END, decoded_data)
        tk.messagebox.showinfo("Decoding", f"Data has been successfully decoded: {decoded_data}")
    else:
        tk.messagebox.showerror("Error", "Please fill in all fields.")

# GUI setup
root = tk.Tk()
root.title("Image Steganography Tool")

frame = ttk.Frame(root, padding="10")
frame.grid(column=0, row=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# Entry widget for input image
label_input_image = ttk.Label(frame, text="Input Image:")
label_input_image.grid(column=0, row=0, sticky=tk.W)

entry_input_image = ttk.Entry(frame, width=40)
entry_input_image.grid(column=1, row=0, sticky=(tk.W, tk.E), padx=10, pady=5)

button_browse_input = ttk.Button(frame, text="Browse", command=lambda: browse_image(entry_input_image))
button_browse_input.grid(column=2, row=0, sticky=tk.W, pady=5)

# Entry widget for output image
label_output_image = ttk.Label(frame, text="Output Image:")
label_output_image.grid(column=0, row=1, sticky=tk.W)

entry_output_image = ttk.Entry(frame, width=40)
entry_output_image.grid(column=1, row=1, sticky=(tk.W, tk.E), padx=10, pady=5)

button_browse_output = ttk.Button(frame, text="Browse", command=lambda: browse_image(entry_output_image))
button_browse_output.grid(column=2, row=1, sticky=tk.W, pady=5)

# Text widget for secret data input
label_secret_data = ttk.Label(frame, text="Enter Secret Data:")
label_secret_data.grid(column=0, row=2, sticky=tk.W)

entry_secret_data = tk.Text(frame, width=40, height=5)
entry_secret_data.grid(column=1, row=2, sticky=(tk.W, tk.E), padx=10, pady=5)

# Buttons for encoding and decoding
button_encode = ttk.Button(frame, text="Encode", command=encode_data)
button_encode.grid(column=0, row=3, sticky=tk.W, pady=10)

button_decode = ttk.Button(frame, text="Decode", command=decode_data)
button_decode.grid(column=1, row=3, sticky=tk.W, pady=10)

root.mainloop()
